MAPLES-DR Intervariability Study
================================

In [1]:
# Import maples-dr
import maples_dr
import numpy as np
import pandas as pd
from coloraide import Color
from IPython.display import display

# Import visualization tools
from ipywidgets import HTML, Dropdown, GridBox, Layout
from jppype import imshow, sync_views, vscode_theme
from maples_dr.dataset import BiomarkerField as Bio
from maples_dr.dataset import FundusField as Fundus
from maples_dr.quick_api import GLOBAL_LOADER
from sklearn.metrics import cohen_kappa_score, confusion_matrix
from tqdm.notebook import tqdm

# Import utilities
from variability_study_utils import (
    centroid,
    load_new_annotations,
    multi_annotator_regions_diff,
    regions_f1,
)

vscode_theme()

Thanks for using MAPLES-DR!
  When using this dataset in academic works,
  please cite: ]8;id=978758;https://arxiv.org/abs/2402.04258\https://arxiv.org/abs/2402.04258]8;;\

HTML(value="<style>\n        .cell-output-ipywidget-background {\n                background: transparent !imp…

In [2]:
maples_dr.configure(
    maples_dr_path="../PATH/TO/MAPLES-DR/AdditionalData.zip",
    messidor_path="../PATH/TO/MESSIDOR/",
    image_format="bgr",
    preprocessing="clahe",
)
maples_dataset = GLOBAL_LOADER.load_dataset("all_with_duplicates")

# Variability study on the duplicated images

In [3]:
duplicates = list(GLOBAL_LOADER.dataset_record["duplicates"].items())

#### Qualitative study

In [4]:
ID = 0
sample1, sample2 = [maples_dataset[_] for _ in duplicates[ID]]

In [5]:
selectors = [
    Dropdown(
        options=[field.value for field in Bio],
        description="Biomarker:",
        layout=Layout(width="auto"),
        value=("brightLesions", "redLesions", "vessels")[i],
    )
    for i in range(3)
]
views = [imshow(sample1["fundus"]) for i in range(3)]
for i in range(1, 3):
    views[i]._left_ruler = False

for i in range(3):

    def set_label(biomarker, i=i):
        if isinstance(biomarker, dict):
            biomarker = biomarker["new"]
        views[i].add_label(
            sample1[biomarker] + 2 * sample2[biomarker],
            colormap={1: "#a56fb9", 2: "#7aa8ba", 3: "white"},
            name="biomarker",
        )

    selectors[i].observe(set_label, "value")

    set_label(selectors[i].value)


sync_views(*views)

GridBox(
    selectors + views,
    layout=Layout(
        grid_template_columns="repeat(3, 1fr)", grid_template_rows="auto 600px"
    ),
)

GridBox(children=(Dropdown(description='Biomarker:', index=4, layout=Layout(width='auto'), options=('opticCup'…

In [6]:
selector = Dropdown(
    options=[field.value for field in Bio],
    description="Biomarker:",
    layout=Layout(width="auto"),
    value=("brightLesions", "redLesions", "vessels")[i],
)

duplicates_samples = [[maples_dataset[_[0]], maples_dataset[_[1]]] for _ in duplicates]
(s1a, s1b), (s2a, s2b) = duplicates_samples
view1 = imshow(s1a["fundus"])
view2 = imshow(s1a["fundus"])
view2._left_ruler = False


def set_label(biomarker):
    if isinstance(biomarker, dict):
        biomarker = biomarker["new"]
    view1.add_label(
        s1a[biomarker] + 2 * s1b[biomarker],
        colormap={1: "#a56fb9", 2: "#7aa8ba", 3: "white"},
        name="biomarker",
    )
    view2.add_label(
        s1a.read_biomarker(biomarker, pre_annotation=True)
        + 2 * s1b.read_biomarker(biomarker, pre_annotation=True),
        colormap={1: "#a56fb9", 2: "#7aa8ba", 3: "white"},
        name="biomarker",
    )


selector.observe(set_label, "value")

set_label(selectors[i].value)

sync_views(view1, view2)

GridBox(
    [selector, HTML(), view1, view2],
    layout=Layout(
        grid_template_columns="repeat(2, 1fr)", grid_template_rows="auto 600px"
    ),
)

GridBox(children=(Dropdown(description='Biomarker:', index=3, layout=Layout(width='auto'), options=('opticCup'…

In [7]:
selector = Dropdown(
    options=[field.value for field in Bio],
    description="Biomarker:",
    layout=Layout(width="auto"),
    value=("brightLesions", "redLesions", "vessels")[i],
)

duplicates_samples = [[maples_dataset[_[0]], maples_dataset[_[1]]] for _ in duplicates]
(s1a, s1b), (s2a, s2b) = duplicates_samples
view1b = imshow(s2a["fundus"])
view2b = imshow(s2a["fundus"])
view2b._left_ruler = False


def set_label(biomarker):
    if isinstance(biomarker, dict):
        biomarker = biomarker["new"]
    view1b.add_label(
        s2a[biomarker] + 2 * s2b[biomarker],
        colormap={1: "#a56fb9", 2: "#7aa8ba", 3: "white"},
        name="biomarker",
    )
    view2b.add_label(
        s2a.read_biomarker(biomarker, pre_annotation=True)
        + 2 * s2b.read_biomarker(biomarker, pre_annotation=True),
        colormap={1: "#a56fb9", 2: "#7aa8ba", 3: "white"},
        name="biomarker",
    )


selector.observe(set_label, "value")

set_label(selectors[i].value)

sync_views(view1b, view2b)

GridBox(
    [selector, HTML(), view1b, view2b],
    layout=Layout(
        grid_template_columns="repeat(2, 1fr)", grid_template_rows="auto 600px"
    ),
)

GridBox(children=(Dropdown(description='Biomarker:', index=3, layout=Layout(width='auto'), options=('opticCup'…

### Quantitative

In [8]:
biomarkers = [
    field.value
    for field in (
        Bio.VESSELS,
        Bio.OPTIC_CUP,
        Bio.OPTIC_DISC,
        Bio.MACULA,
        Bio.RED_LESIONS,
        Bio.BRIGHT_LESIONS,
    )
]

data = {}
for bio in biomarkers:
    bio1 = np.array([s1a[bio], s2a[bio]], dtype=bool)
    bio2 = np.array([s1b[bio], s2b[bio]], dtype=bool)
    kappa = cohen_kappa_score(bio1.flatten(), bio2.flatten(), labels=[0, 1])
    accuracy = np.mean(bio1 == bio2)
    dice = 2 * np.sum(bio1 * bio2) / (np.sum(bio1) + np.sum(bio2))
    data[bio] = {"kappa": kappa, "accuracy": accuracy, "dice": dice}

for bio in [Bio.MACULA, Bio.OPTIC_CUP, Bio.OPTIC_DISC]:
    d = np.mean(
        [centroid(sa[bio]).distance(centroid(sb[bio])) for sa, sb in duplicates_samples]
    )
    data[bio.value]["distance"] = d

for bio in [Bio.RED_LESIONS, Bio.BRIGHT_LESIONS]:
    data[bio.value]["mean detection f1"] = np.nanmean(
        [regions_f1(sa[bio], sb[bio]) for sa, sb in duplicates_samples]
    )


pd.DataFrame(data).round(3).fillna("")

,vessels,opticCup,opticDisc,macula,redLesions,brightLesions
kappa,0.795,0.847,0.958,0.045,0.48,0.176
accuracy,0.961,0.999,0.999,0.999,0.999,0.998
dice,0.817,0.848,0.958,0.045,0.48,0.177
distance,,5.581,2.365,24.316,,
mean detection f1,,,,,0.616,0.083


# Variability Study on reannotated Images

In [3]:
new_annotations = load_new_annotations()
N_annotators = len(new_annotations)
published_dataset = GLOBAL_LOADER.load_dataset()

### Visualize the data

In [30]:
images = new_annotations[0].keys()
current_img = [images[0], 0]

In [32]:
img_selector = Dropdown(
    options=images,
    description="Biomarker:",
    layout=Layout(width="auto"),
    value=current_img[0],
)

retinologist_selector = Dropdown(
    options=[i + 1 for i in range(len(new_annotations))],
    description="Retinologist:",
    layout=Layout(width="auto"),
    value=current_img[1] + 1,
)

s = published_dataset[images[0]]

view1 = imshow(s["fundus"])
view2 = imshow(s["fundus"])
view2._left_ruler = False
view3 = imshow(s["fundus"])
view3._top_ruler = False
view4 = imshow(s["fundus"])
view4._top_ruler = False
view4._left_ruler = False

cmap = {
    # --- Pre-annotations Edited---
    2: "#d41616",  # Not in published, removed from pre-annotation
    3: "#860e0e",  # In published, removed from pre-annotation
    4: "#154f00",  # Not in published nor pre-annotation, added by ret
    5: "#248600",  # Published not in pre-annotation, added by ret
    # --- Pre-annotations not edited ---
    1: "#a8394988",  # In published, not in refined
    6: "#999bc9",  # Not in published
    7: "#cccccc",  # In published and refined
}


def setup_views():
    img, ret = current_img

    s_published = published_dataset[img]
    s_refined = new_annotations[ret][img]

    for v, l in zip(
        [view1, view2, view3, view4],
        [Bio.MICROANEURYSMS, Bio.HEMORRHAGES, Bio.EXUDATES, Bio.COTTON_WOOL_SPOTS],
        strict=True,
    ):
        diff = (
            s_published[l]
            + 2 * s_refined.read_biomarker(l, pre_annotation=True)
            + 4 * s_refined.read_biomarker(l, pre_annotation=False)
        )
        v["background"] = s_published["fundus"]
        v.add_label(
            diff,
            name="diff",
            colormap=cmap,
        )


def set_img(img):
    if isinstance(img, dict):
        img = img["new"]
    current_img[0] = img
    setup_views()


def set_ret(ret):
    if isinstance(ret, dict):
        ret = ret["new"]
    current_img[1] = ret - 1
    setup_views()


img_selector.observe(set_img, "value")
retinologist_selector.observe(set_ret, "value")

setup_views()
sync_views(view1, view2, view3, view4)

legend1 = HTML(
    f"""
<h3 style="color: var(--jppype-foreground-color);"> Pre-annotations edition </h3>
<p style="color: var(--jppype-foreground-color); font-size: 14px; text-align: center;">
    Removed by retinologist:
    <span style="display: inline-block"><span style="background-color: {cmap[2]}; width: 14px; height: 14px; 
    border-radius: 7px; display: inline-block; margin-left: 20px"> </span> Not published; </span>
    <span style="display: inline-block"><span style="background-color: {cmap[3]}; width: 14px; height: 14px; 
    border-radius: 7px; display: inline-block; margin-left: 20px"> </span> in published </span>
</p>
<p style="color: var(--jppype-foreground-color); font-size: 14px; text-align: center;">    
    Added by retinologist:
    <span style="display: inline-block"><span style="background-color: {cmap[4]}; width: 14px; height: 14px;
     border-radius: 7px; display: inline-block; margin-left: 20px"> </span> Not in published; </span>
    <span style="display: inline-block"><span style="background-color: {cmap[5]}; width: 14px; height: 14px; 
    border-radius: 7px; display: inline-block; margin-left: 20px"> </span> In published. </span>
</p>
"""
)
legend2 = HTML(
    f"""
<h3 style="color: var(--jppype-foreground-color);"> Pre-annotations kept </h3>
<p  style="color: var(--jppype-foreground-color); font-size: 14px; text-align: center;">
    <span style="display: inline-block"><span style="background-color: {cmap[1]}; width: 14px; height: 14px; border-radius: 7px; display: inline-block; margin-left: 20px"> </span> Only in published; </span>

    <span style="display: inline-block"><span style="background-color: {cmap[6]}; width: 14px; height: 14px; border-radius: 7px; display: inline-block; margin-left: 20px"> </span> Absent in published; </span>

    <span style="display: inline-block"><span style="background-color: {cmap[7]}; width: 14px; height: 14px; border-radius: 7px; display: inline-block; margin-left: 20px"> </span> In all 3. </span>
</p>
"""
)

GridBox(
    [img_selector, retinologist_selector, legend1, legend2, view1, view2, view3, view4],
    layout=Layout(
        grid_template_columns="repeat(2, 1fr)",
        grid_template_rows="auto auto 415px 400px",
    ),
)

GridBox(children=(Dropdown(description='Biomarker:', layout=Layout(width='auto'), options=('20060522_46266_010…

## Qualitative analysis

In [4]:
images = new_annotations[0].keys()
current_img2 = [images[0], Bio.RED_LESIONS.value]

In [37]:
img_selector2 = Dropdown(
    options=images,
    description="Biomarker:",
    layout=Layout(width="auto"),
    value=current_img2[0],
)

biomarker_selector = Dropdown(
    options=[
        _.value
        for _ in [
            Bio.MICROANEURYSMS,
            Bio.HEMORRHAGES,
            Bio.EXUDATES,
            Bio.COTTON_WOOL_SPOTS,
            Bio.RED_LESIONS,
            Bio.BRIGHT_LESIONS,
        ]
    ],
    description="Biomarker:",
    layout=Layout(width="auto"),
    value=current_img2[1],
)

s = published_dataset[images[0]]

view6 = imshow(s["fundus"])
view7 = imshow(s["fundus"])
view7._left_ruler = False
view8 = imshow(s["fundus"])
view8._left_ruler = False

cmap = {
    # --- Pre-annotations Edited---
    1: "red",  # Removed
    2: "green",  # Added
    # --- Pre-annotations not edited ---
    3: "white",
}


def setup_views2():
    img, bio = current_img2

    for s, v in zip([_[img] for _ in new_annotations], [view6, view7, view8]):
        v["background"] = s["fundus"]
        diff = s.read_biomarker(bio, pre_annotation=True) + 2 * s.read_biomarker(
            bio, pre_annotation=False
        )
        v.add_label(
            diff,
            name="diff",
            colormap=cmap,
        )


def set_img2(img):
    if isinstance(img, dict):
        img = img["new"]
    current_img2[0] = img
    setup_views2()


def set_bio2(bio):
    if isinstance(bio, dict):
        bio = bio["new"]
    current_img2[1] = bio
    setup_views2()


img_selector2.observe(set_img2, "value")
biomarker_selector.observe(set_bio2, "value")

setup_views2()
sync_views(view6, view7, view8)

legend1 = HTML(
    f"""
<p style="color: var(--jppype-foreground-color); font-size: 14px; text-align: center;">
    <span style="display: inline-block"><span style="background-color: {cmap[2]}; width: 14px; height: 14px; 
    border-radius: 7px; display: inline-block; margin-left: 20px"> </span> Added </span>
    <span style="display: inline-block"><span style="background-color: {cmap[1]}; width: 14px; height: 14px; 
    border-radius: 7px; display: inline-block; margin-left: 20px"> </span> Removed </span>
</p>
<h3 style="color: var(--jppype-foreground-color);"> Daniel </h3>
"""
)
legend2 = HTML('<h3 style="color: var(--jppype-foreground-color);"> Fares </h3>')
legend3 = HTML('<h3 style="color: var(--jppype-foreground-color);"> Marie Carole </h3>')

GridBox(
    [
        img_selector2,
        biomarker_selector,
        HTML(),
        legend1,
        legend2,
        legend3,
        view6,
        view7,
        view8,
    ],
    layout=Layout(
        grid_template_columns="repeat(3, 1fr)",
        grid_template_rows="auto auto 500px",
    ),
)

GridBox(children=(Dropdown(description='Biomarker:', layout=Layout(width='auto'), options=('20060522_46266_010…

## Quantitative analysis

In [4]:
def format_coef(cac_result):
    coef = cac_result["est"]["coefficient_value"]
    ci = cac_result["est"]["confidence_interval"]
    return f"{coef:.3f} (CI: {ci[0]:.3f}-{ci[1]:.3f})"


def majority_voting(samples: list):
    return np.mean(samples, axis=0) > 0.5

### Segmentation Multi-graders

In [14]:
data_red = []
data_bright = []

red_labels = {1: Bio.MICROANEURYSMS, 2: Bio.HEMORRHAGES}
bright_labels = {1: Bio.EXUDATES, 2: Bio.COTTON_WOOL_SPOTS, 3: Bio.DRUSENS}

for samples in tqdm(zip(*new_annotations, strict=True), total=16):
    mask = samples[0].read_roi_mask()
    data_red += [np.stack([s.read_multiple_biomarkers(red_labels)[mask] for s in samples])]
    data_bright += [np.stack([s.read_multiple_biomarkers(bright_labels)[mask] for s in samples])]


def compute_multigrader_stats(flatten_data):
    from irrCAC.raw import CAC

    flatten_data = pd.DataFrame(np.concatenate(flatten_data, axis=1).T)
    cac = CAC(flatten_data, digits=3)
    return {
        "Fleiss's Kappa": format_coef(cac.fleiss()),
        "Gwet's AC1": format_coef(cac.gwet()),
    }


stats_bright = compute_multigrader_stats(data_bright)
stats_red = compute_multigrader_stats(data_red)

pd.DataFrame({"Red Lesions": stats_red, "Bright Lesions": stats_bright})

  0%|          | 0/16 [00:00<?, ?it/s]

,Red Lesions,Bright Lesions
Fleiss's Kappa,0.947 (CI: 0.946-0.948),0.897 (CI: 0.896-0.899)
Gwet's AC1,1.000 (CI: 1.000-1.000),0.999 (CI: 0.999-0.999)


### Segmentation Graders vs Majority vote (CAC)

In [10]:
red_labels = {1: Bio.MICROANEURYSMS, 2: Bio.HEMORRHAGES}
N_red = len(red_labels) + 1
red_cm = [np.zeros((N_red,) * 2) for _ in range(N_annotators)]

bright_labels = {1: Bio.EXUDATES, 2: Bio.COTTON_WOOL_SPOTS, 3: Bio.DRUSENS}
N_bright = len(bright_labels) + 1
bright_cm = [np.zeros((N_bright,) * 2) for _ in range(N_annotators)]

for samples in tqdm(zip(*new_annotations, strict=True), total=16):
    mask = samples[0].read_roi_mask()

    red = [s.read_multiple_biomarkers(red_labels) for s in samples]
    mv = majority_voting(red)[mask]
    for i, s in enumerate(red):
        red_cm[i] += confusion_matrix(s[mask], mv, labels=list(range(N_red)))

    bright = [s.read_multiple_biomarkers(bright_labels) for s in samples]
    mv = majority_voting(bright)[mask]
    for i, s in enumerate(bright):
        bright_cm[i] += confusion_matrix(s[mask], mv, labels=list(range(N_bright)))

  0%|          | 0/16 [00:00<?, ?it/s]

In [11]:
def compute_graders_stats(cm):
    from irrCAC.table import CAC

    stats = pd.DataFrame(columns=("Cohen's Kappa", "Gwet's AC1", "Agreement"))

    for expert, cm in zip(["Daniel", "Fares", "Marie Carole"], cm, strict=True):
        cac = CAC(pd.DataFrame(cm), digits=3)
        stats.loc[expert] = {
            "Cohen's Kappa": format_coef(cac.cohen()),
            "Gwet's AC1": format_coef(cac.gwet()),
            "Agreement": format_coef(cac.pa2()),
        }
    return stats


print("=== BRIGHT LESIONS ===")
display(compute_graders_stats(bright_cm))

print("=== RED LESIONS ===")
display(compute_graders_stats(red_cm))

=== BRIGHT LESIONS ===


,Cohen's Kappa,Gwet's AC1,Agreement
Daniel,0.868 (CI: 0.866-0.870),0.999 (CI: 0.999-0.999),0.999 (CI: 0.999-0.999)
Fares,0.886 (CI: 0.885-0.888),0.999 (CI: 0.999-0.999),0.999 (CI: 0.999-0.999)
Marie Carole,0.902 (CI: 0.901-0.904),0.999 (CI: 0.999-0.999),0.999 (CI: 0.999-0.999)


=== RED LESIONS ===


,Cohen's Kappa,Gwet's AC1,Agreement
Daniel,0.789 (CI: 0.787-0.791),0.999 (CI: 0.999-0.999),0.999 (CI: 0.999-0.999)
Fares,0.789 (CI: 0.787-0.791),0.999 (CI: 0.999-0.999),0.999 (CI: 0.999-0.999)
Marie Carole,0.798 (CI: 0.796-0.800),0.999 (CI: 0.999-0.999),0.999 (CI: 0.999-0.999)


### Segmentation Graders vs Majority Voting (average, std)

In [10]:
biomarkers = [
    field.value
    for field in (
        Bio.MICROANEURYSMS,
        Bio.HEMORRHAGES,
        Bio.RED_LESIONS,
        Bio.EXUDATES,
        Bio.COTTON_WOOL_SPOTS,
        Bio.BRIGHT_LESIONS,
    )
]

stats_per_ret = {"Fares": {}, "Daniel": {}, "Marie Carole": {}}

for samples in zip(*new_annotations, strict=True):
    mask = samples[0].read_roi_mask()

    for bio in biomarkers:
        mv_img = majority_voting([s[bio] for s in samples])
        mv = mv_img[mask]

        for ret, sample in zip(stats_per_ret.keys(), samples, strict=True):
            s_img = sample[bio]
            s = s_img[mask]

            if np.sum(s) + np.sum(mv) > 0:
                dice = 2 * np.sum(s * mv) / (np.sum(s) + np.sum(mv))
            else:
                dice = float("nan")

            stats = {
                "dice": dice,
                "f1 detection": regions_f1(s_img, mv_img),
            }
            for k, v in stats.items():
                stats_per_ret[ret].setdefault(bio, {}).setdefault(k, []).append(v)

In [11]:
def format_mean_std(stats):
    if isinstance(stats, dict):
        return {k: format_mean_std(v) for k, v in stats.items()}
    stats = np.array(stats)
    stats = stats[~np.isnan(stats)]
    return (
        f"{np.mean(stats):.3f} "
        + ("" if len(stats) == 1 else f"(±{np.std(stats):.3f})")
        if len(stats)
        else ""
    )


def select(stats, stat_name):
    return {
        ret: {bio: stats[ret][bio][stat_name] for bio in stats[ret]} for ret in stats
    }


formatted_stats = format_mean_std(stats_per_ret)

print("=== DICE ===")
display(pd.DataFrame(select(formatted_stats, "dice")).T)

print("=== F1 DETECTION ===")
display(pd.DataFrame(select(formatted_stats, "f1 detection")).T)

=== DICE ===


,microaneurysms,hemorrhages,redLesions,exudates,cottonWoolSpots,brightLesions
Fares,0.976 (±0.055),0.736 (±0.395),0.981 (±0.026),0.689 (±0.356),0.756 (±0.423),0.736 (±0.292)
Daniel,0.971 (±0.060),0.995 (±0.016),0.971 (±0.059),0.880 (±0.283),0.250 (±0.433),0.752 (±0.396)
Marie Carole,0.926 (±0.240),0.763 (±0.334),0.896 (±0.248),0.855 (±0.292),0.625 (±0.451),0.649 (±0.410)


=== F1 DETECTION ===


,microaneurysms,hemorrhages,redLesions,exudates,cottonWoolSpots,brightLesions
Fares,0.984 (±0.039),0.854 (±0.302),0.996 (±0.011),0.725 (±0.352),0.821 (±0.309),0.837 (±0.269)
Daniel,0.971 (±0.051),0.993 (±0.020),0.971 (±0.050),0.918 (±0.179),0.250 (±0.433),0.822 (±0.307)
Marie Carole,0.923 (±0.239),0.743 (±0.311),0.904 (±0.237),0.871 (±0.285),0.583 (±0.449),0.717 (±0.395)


### Edition Comparison

In [5]:
def multi_diff(sample_ID, bio):
    samples = [_[sample_ID] for _ in new_annotations]
    s1 = samples[0]
    labels, diffs = multi_annotator_regions_diff(
        s1.read_biomarker(bio, pre_annotation=True),
        *[sample[bio] for sample in samples],
        change_max_iou=0.75,
    )
    v = imshow(s1["fundus"])

    def diff_to_color(diff):
        if "A" in diff:
            return (
                Color.interpolate(["yellow", "green"])(
                    sum(_ == "A" for _ in diff) / len(diff)
                )
                .convert("srgb")
                .to_string(hex=True)
            )
        return (
            Color.average(
                [
                    Color.interpolate(["white", "blue"], space="hsv")(
                        (sum(_ == "C" for _ in diff) / len(diff)) ** 0.2
                    ),
                    Color.interpolate(
                        ["white", "red"],
                    )((sum(_ == "R" for _ in diff) / len(diff)) ** 0.2),
                ],
                space="hsv",
            )
            .convert("srgb")
            .to_string(hex=True)
        )

    v.add_label(
        labels,
        name="diff",
        colormap={int(k): diff_to_color(v) for k, v in diffs.items()},
    )
    return diffs, v


diffs, v = multi_diff("20060522_46266_0100_PP", Bio.RED_LESIONS)
v

View2D()

In [12]:
lesion_diffs = {}

for samples in zip(*new_annotations):
    for bio in [
        Bio.MICROANEURYSMS,
        Bio.HEMORRHAGES,
        Bio.RED_LESIONS,
        Bio.EXUDATES,
        Bio.COTTON_WOOL_SPOTS,
        Bio.DRUSENS,
        Bio.BRIGHT_LESIONS,
    ]:
        pre_annotation = samples[0].read_biomarker(bio, pre_annotation=True)
        _, diffs = multi_annotator_regions_diff(
            pre_annotation,
            *[s[bio] for s in samples],
        )

        lesion_diffs.setdefault(bio.value, []).extend(list(diffs.values()))

In [14]:
def aggregate_diffs(diffs):
    added = {r + 1: 0 for r in reversed(range(N_annotators))}
    removed = {r + 1: 0 for r in reversed(range(N_annotators))}
    no_change = 0
    for d in diffs:
        if "A" in d:
            added[sum(_ == "A" for _ in d)] += 1
        elif "R" in d:
            removed[sum(_ == "R" for _ in d)] += 1
        else:
            no_change += 1

    print(removed, no_change, sum(removed) + no_change)

    return {
        ("Preannotated", ""): sum(removed.values()) + no_change,
        ("Removed", "1/3 ann"): removed[1],
        ("Removed", "2/3 ann"): removed[2],
        ("Removed", "all ann"): removed[3],
        ("No change", ""): f"{no_change} ({no_change/(sum(removed) + no_change):.0%})",
        ("Added", "1/3 ann"): added[1],
        ("Added", "2/3 ann"): added[2],
        ("Added", "all ann"): added[3],
    }


df = pd.DataFrame(
    {lesions: aggregate_diffs(diffs) for lesions, diffs in lesion_diffs.items()}
).T
display(df)

{3: 0, 2: 2, 1: 10} 369 375
{3: 0, 2: 4, 1: 3} 40 46
{3: 0, 2: 3, 1: 8} 383 389
{3: 0, 2: 1, 1: 12} 169 175
{3: 1, 2: 4, 1: 8} 1 7
{3: 0, 2: 0, 1: 0} 0 6
{3: 1, 2: 5, 1: 18} 172 178


Preannotated Removed                  No change   Added  \
                             1/3 ann 2/3 ann all ann            1/3 ann   
microaneurysms           381      10       2       0  369 (98%)      13   
hemorrhages               47       3       4       0   40 (87%)      21   
redLesions               394       8       3       0  383 (98%)      33   
exudates                 182      12       1       0  169 (97%)      25   
cottonWoolSpots           14       8       4       1    1 (14%)       1   
drusens                    0       0       0       0     0 (0%)      17   
brightLesions            196      18       5       1  172 (97%)      32   

                                 
                2/3 ann all ann  
microaneurysms        0       0  
hemorrhages           0       0  
redLesions            0       0  
exudates              0       0  
cottonWoolSpots       0       0  
drusens               0       0  
brightLesions         1       0